*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [95]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [96]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

zsh:1: command not found: wget
Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [97]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [98]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [99]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [100]:
print("shape of df_books is ", df_books.shape)
print("shape of df_ratings is ", df_ratings.shape)

shape of df_books is  (271379, 3)
shape of df_ratings is  (1149780, 3)


In [ ]:
# add your code here - consider creating a new cell for each section of code

In [108]:
#get a dataframe with book info and number of ratings
#we'll need this for our final function (for error message for too few ratings for a book)

#get ratings per isbn
ratings_per_isbn = df_ratings.groupby(['isbn'])['user'].count().reset_index()
ratings_per_isbn.columns = ['isbn','num_ratings']
ratings_per_isbn.sort_values(by='num_ratings', ascending=False)
ratings_per_isbn

#merge this into df_books
combined = df_books.merge(ratings_per_isbn, how='left').sort_values(by='num_ratings')
combined

,isbn,title,author,num_ratings
0,0195153448,Classical Mythology,Mark P. O. Morford,1.0
178109,1564743861,Never Fade Away: A Novel,William Hart,1.0
178110,0060537256,Phantom Stallion #7: Desert Dancer (Phantom St...,Terri Farley,1.0
178111,1890626465,Feminist Fantasies,Phyllis Schlafly,1.0
178112,0307126218,The Purrfect Crime (Golden Look-Look Book),Andrew Helfer,1.0
...,...,...,...,...
271360,1874166633,Introducing Nietzsche (Foundations in Children...,Laurence Gane,NaN
271361,0130897930,Core Web Programming (2nd Edition),Marty Hall,NaN
271362,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,NaN
271363,2268032019,Petite histoire de la dÃ?Â©sinformation,Vladimir Volkoff,NaN


In [112]:
#number of ratings per isbn in df_ratings
num_ratings_per_isbn = df_ratings.groupby(["isbn"]).count().reset_index()
num_ratings_per_isbn.head()

,isbn,user,rating
0,0330299891,2,2
1,0375404120,2,2
2,0586045007,1,1
3,9022906116,2,2
4,9032803328,1,1


In [113]:
#get list of ISBNs with 100 or more ratings
ISBNs_w_100_or_more_ratings = num_ratings_per_isbn.loc[num_ratings_per_isbn["rating"] > 99]["isbn"]
ISBNs_w_100_or_more_ratings.head()

3800    002542730X
5416    0060008032
5683    0060096195
6204    006016848X
6338    0060173289
Name: isbn, dtype: object

In [114]:
#get books that have no less than 100 ratings
books_w_100_or_more_ratings = df_books.loc[df_books["isbn"].isin(ISBNs_w_100_or_more_ratings)]
books_w_100_or_more_ratings.head()

,isbn,title,author
18,0440234743,The Testament,John Grisham
19,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison
26,0971880107,Wild Animus,Rich Shapero
27,0345402871,Airframe,Michael Crichton
28,0345417623,Timeline,MICHAEL CRICHTON


In [115]:
#get the number of ratings per user
num_ratings_per_user = df_ratings[["user", "rating"]].groupby(["user"]).count().reset_index()
num_ratings_per_user.head()


,user,rating
0,2,1
1,7,1
2,8,18
3,9,3
4,10,2


In [116]:
#get a list of users with 200 or more ratings
users_w_200_or_more_ratings = num_ratings_per_user.loc[num_ratings_per_user["rating"] > 199]["user"]
users_w_200_or_more_ratings.head()

95       254
791     2276
981     2766
1049    2977
1177    3363
Name: user, dtype: int64

In [117]:
#filter out unqualified users and books
qualified_ratings_for_pivot = df_ratings.loc[df_ratings["user"].isin(users_w_200_or_more_ratings)]
qualified_ratings_for_pivot = qualified_ratings_for_pivot.loc[qualified_ratings_for_pivot["isbn"].isin(books_w_100_or_more_ratings["isbn"])]
qualified_ratings_for_pivot.head()

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0


In [59]:
# pivot ratings into book features
df_ratings_pivoted = qualified_ratings_for_pivot.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

In [60]:
df_ratings_pivoted.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
df_ratings_pivoted.shape

(727, 888)

In [119]:
#For more efficient calculation and less memory footprint, we need to transform the values of the dataframe into a scipy sparse matrix.
# convert dataframe of book features to scipy sparse matrix
# this part requires a lot of memory!
books_with_ratings_matrix = csr_matrix(df_ratings_pivoted.values)

In [63]:
books_with_ratings_matrix

<727x888 sparse matrix of type '<class 'numpy.float32'>'
	with 12548 stored elements in Compressed Sparse Row format>

In [120]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [121]:
model_knn = NearestNeighbors(metric='cosine')
model_knn.fit(books_with_ratings_matrix)

NearestNeighbors(metric='cosine')

In [88]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    #return error if book is not there
    try:
        book_info = books_w_100_or_more_ratings.loc[books_w_100_or_more_ratings['title'] == book]
    except KeyError:
        print('Too few reviews (or other error) for the book')
        return
    
    distance, indice = model_knn.kneighbors(df_ratings_pivoted.loc[df_ratings_pivoted.index.isin(book_info["isbn"]),:].values.reshape(1,-1))
    
    distance =  distance[0][1:]
    indice = indice[0][1:]
    
    book_titles = [
    df_books.loc[df_books['isbn']==df_ratings_pivoted.iloc[i].name]["title"].values[0]
    for i in indice
    ]
    
    recommended = [list(z) for z in zip(book_titles, distance)][::-1]
    
    return [book, recommended]

In [122]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    are_there_enough_ratings = combined.loc[combined['title'] == book]
    #return error if book is not there
    if are_there_enough_ratings['num_ratings'].item() <100:
        return print('Too few reviews (or other error) for the book')
    book_info = books_w_100_or_more_ratings.loc[books_w_100_or_more_ratings['title'] == book]
    
    distance, indice = model_knn.kneighbors(df_ratings_pivoted.loc[df_ratings_pivoted.index.isin(book_info["isbn"]),:].values.reshape(1,-1))
    
    distance =  distance[0][1:]
    indice = indice[0][1:]
    
    book_titles = [
    df_books.loc[df_books['isbn']==df_ratings_pivoted.iloc[i].name]["title"].values[0]
    for i in indice
    ]
    
    recommended = [list(z) for z in zip(book_titles, distance)][::-1]
    
    return [book, recommended]

In [124]:
get_recommends("Feminist Fantasies")

Too few reviews (or other error) for the book


In [127]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075],
  ['The Lovely Bones: A Novel', 0.7234864]]]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [128]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
